In [ ]:
def run(script='TEMP3D_default.mlx', log=None, ml_log=None,
        mlp_in=None, mlp_out=None, overwrite=False, file_in=None,
        file_out=None, output_mask=None, cmd=None, ml_version=ML_VERSION,
        print_meshlabserver_output=True):
    """Run meshlabserver in a subprocess.
    Args:
        log (str): filename of the log file for meshlabxml. If not
            None, all meshlabserver stdout and stderr messages
            will be appended to this file.
        ml_log (str): filename of the log file output directly by
            meshlabserver.
        mlp_in (str or list): input meshlab project file. Can be a  
            single filename or a list of filenames. Filenames will
            be loaded in the order given. All project files will
            be loaded before individual input files. If you want
            to load project and input files in a different order
            then you should use a custom cmd.
        mlp_out (str): output meshlab project file. Specify a   
            single filename (meshlabserver accepts multiple output
            project filenames, however they will all be identical,
            so there is little use). When this option is used all
            layers will be saved as ply files.
        overwrite (bool): when specifying mlp_out, this determines
            whether any existing files will be overwritten (if
            True) or new filenames created (if False). If a new
            project file is created meshes will have '_out' added
            to their name.
        file_in (str or list): input mesh filename. Can be a single
            filename or a list of filenames. Filenames will be
            loaded in the order given. All project files will be
            loaded before individual input files. If you want to
            load project and input files in a different order then
            you should use a custom cmd.
        file_out (str or list): output mesh filename. Can be a
            single filename or a list of filenames. The current
            layer will be saved to this filename or filenames.
            Multiple filenames are useful for saving to multiple
            formats at the same time. Currently there is no way to
            output multiple layers except for saving a mlp project
            file.
        output_mask (str or list): output mask options for the
            output file. Values must include the flag, i.e. -m or
            -output_mask. If this is not provided for an output
            file then function "default_output_mask" is used to
            determine default values.
        script (str): the mlx filter script filename to execute.
        cmd (str): a full meshlabserver command line, such as
            "meshlabserver -input file.stl". If not None, this
            will override all other arguements except for log.
        print_meshlabserver_output (bool): Pass meshlabserver's output to stdout; useful for debugging.
                                           Only used if log is None.
    Notes:
        Meshlabserver can't handle spaces in paths or filenames (on Windows at least; haven't tested on other platforms). Enclosing the name in quotes or escaping the space has no effect.
    Returns:
        return code of meshlabserver process; 0 if successful
    """
    if cmd is None:
        cmd = 'meshlabserver'
        if ml_log is not None:
            # Initialize ml_log
            ml_log_file = open(ml_log, 'w')
            ml_log_file.close()
            cmd += ' -l %s' % ml_log
        if mlp_in is not None:
            # make a list if it isn't already
            mlp_in = util.make_list(mlp_in)
            for val in mlp_in:
                cmd += ' -p "%s"' % val
        if mlp_out is not None:
            cmd += ' -w %s' % mlp_out
            if overwrite:
                cmd += ' -v'
        if (mlp_in is None) and (file_in is None):
			# If no input files are provided use the default created by begin().
			# This works around the fact that meshlabserver will
			# not run without an input file.
            file_in = ['TEMP3D.xyz']
        if file_in is not None:
            # make a list if it isn't already
            file_in = util.make_list(file_in)
            for val in file_in:
                if val == 'bunny':
                    cmd += ' -i "%s"' % os.path.join(THIS_MODULEPATH, os.pardir,
                                                     'models', 'bunny_flat(1Z).ply')
                elif val == 'bunny_raw':
                    cmd += ' -i "%s"' % os.path.join(THIS_MODULEPATH, os.pardir,
                                                     'models', 'bunny_raw(-1250Y).ply')
                else:
                    cmd += ' -i "%s"' % val
        if file_out is not None:
            # make a list if it isn't already
            file_out = util.make_list(file_out)
            if output_mask is not None:
                output_mask = util.make_list(output_mask)
            else:
                output_mask = []
            for index, val in enumerate(file_out):
                cmd += ' -o "%s"' % val
                try:
                    cmd += ' %s' % output_mask[index]
                except IndexError:  # If output_mask can't be found use defaults
                    cmd += ' %s' % default_output_mask(val, ml_version=ml_version)
        if script is not None:
            cmd += ' -s "%s"' % script
    if log is not None:
        log_file = open(log, 'a')
        log_file.write('meshlabserver cmd = %s\n' % cmd)
        log_file.write('***START OF MESHLAB STDOUT & STDERR***\n')
        log_file.close()
        log_file = open(log, 'a')
    else:
        if print_meshlabserver_output:
            log_file = None
            print('meshlabserver cmd = %s' % cmd)
            print('***START OF MESHLAB STDOUT & STDERR***')
        else:
            log_file = open(os.devnull, 'w')
    while True:
        # TODO: test if shell=True is really needed
        return_code = subprocess.call(cmd, shell=True,
                                      stdout=log_file, stderr=log_file,
                                      universal_newlines=True)
        if log is not None:
            log_file.close()
        if (return_code == 0) or handle_error(program_name='MeshLab', cmd=cmd, log=log):
            break
    if log is not None:
        log_file = open(log, 'a')
        log_file.write('***END OF MESHLAB STDOUT & STDERR***\n')
        log_file.write('meshlabserver return code = %s\n\n' % return_code)
        log_file.close()
    return return_code

In [5]:
#my algorithm
""" ***setup commands need to already have been run*****
1) Get the script name, input mesh file name and output mesh file name
2) Run the command as a subprocess

OR 

1) Get the script name, input mesh file name and output mesh file name
2) Create a copy and append to a bash script that has the setup for meshalab script
3) Add the meshlabserver command
4) Run the command as a subprocess

The setup commands:
#!/usr/bin/env bash
export PATH=${PATH}:/meshlab/src/distrib

function meshlabserver() { xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@; }
export -f meshlabserver

"""

def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result



    
    
    

In [6]:
result = run_meshlab_script("/notebooks/Pymeshfix_vs_Voxelization/remeshing_script_vox.mlx",
                         "/notebooks/Pymeshfix_vs_Voxelization/temp/neuron_648518346341366885_orig.off",
                        "/notebooks/Pymeshfix_vs_Voxelization/temp/neuron_648518346349504955_pymesh_vox_default_mesh_py.off")
print(result)


CompletedProcess(args='xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Pymeshfix_vs_Voxelization/temp/neuron_648518346341366885_orig.off -o /notebooks/Pymeshfix_vs_Voxelization/temp/neuron_648518346349504955_pymesh_vox_default_mesh_py.off -s /notebooks/Pymeshfix_vs_Voxelization/remeshing_script_vox.mlx', returncode=0)


In [7]:
result = run_meshlab_script("remeshing_script_vox.mlx",
                         "temp/neuron_648518346341366885_orig.off",
                        "neuron_648518346349504955_pymesh_vox_default_mesh_py.off")
print(result)

CompletedProcess(args='xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i temp/neuron_648518346341366885_orig.off -o neuron_648518346349504955_pymesh_vox_default_mesh_py.off -s remeshing_script_vox.mlx', returncode=0)


In [ ]:
"""  command 
/usr/bin/meshlabserver -i /notebooks/Pymeshfix_vs_Voxelization/temp/neuron_648518346341366885_orig.off -o /notebooks/Pymeshfix_vs_Voxelization/temp/neuron_648518346349504955_pymesh_vox_default_mesh_py.off -s /notebooks/Pymeshfix_vs_Voxelization/remeshing_script_vox.mlx

"""

In [ ]:
commands_to_run = ['function meshlabserver() { xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@; }',
                  'export -f meshlabserver']

path = 'neuron-' + str(segment_id) + '.sh'
with open(path, 'w+') as f:           # umask in effect when file is created
    f.write('#!/bin/bash\n')
    f.write('echo "hello world"\n')
    f.write(commands_to_run[0]+"\n")
    f.write(commands_to_run[1]+"\n")
    f.write(new_string)

make_executable(path)
os.system("./"+path)

In [ ]:
def make_executable(path):
    mode = os.stat(path).st_mode
    mode |= (mode & 0o444) >> 2    # copy R bits to X
    os.chmod(path, mode)

In [ ]:
#try running a meshlabserver command write here
new_string = ("meshlabserver -i ./temp/neuron_" + str(segment_id) +
           ".off -o ./temp/neuron_" + str(segment_id) + "_new.off " + 
           "-s remeshing_script_5.mlx")
#os.system(new_string)
#print(new_string)